# Plot model output
## Explore NetCDF output variables produced by a successful model run
This notebook is heavily inspired by https://github.com/NCAR/ctsm_python_gallery, https://mpaiao.github.io/FATES_Utils/index.html, and https://fates-users-guide.readthedocs.io/en/latest/index.html.

**Read the instructions carefully. Unless stated otherwise, make sure to execute all code cells in sequential order.**

In [ ]:
import xarray as xr  # NetCDF data handling
import matplotlib.pyplot as plt  # Plotting
import glob  # Wild card file pattern matching and retrieval
import time  # Keeping track of runtime

---
#### Define names and path to where the NetCDF output is located.

In [ ]:
# CHANGE THE TWO FOLLOWING VARIABLES
case_name = "ALP3"  # Same as shown in GUI title (ALP1, BOR2, etc.), used for labelling and file names
case_id = "c6f674050f339d9994a1a51a9362ae9a"  # Copy paste the case ID for the experiment you want to analyze here

# Next line will be set automatically
output_path_str = f"../../cases/{case_id}/archive/lnd/hist/"  # Default path to land output for an experiment

#### Read NetCDF files, combine all files in the directory.
<h5 style="color:red;">NB! Did you include more than one "history tape"?</h5>

This is only relevant if you specified additional output for the fields `hist_fincl2` - `hist_fincl6` in the "NAMELIST - CLM - HISTORY" tab when you created the case in the GUI. There is no automated code included in this notebook to deal with this additional output, but you can easily adapt the cells according to your needs. See the commented out code below (the lines between the triple quotations) for examples on how to read in data from different history tapes.

In [ ]:
# This process can take several minutes depending on the amount and size of files (ca. 5 mins for 10 year monthly default output)
start = time.time()

h0_file_names = glob.glob(output_path_str+"/*h0*.nc")
"""
h1_file_names = glob.glob(output_path_str+"/*h1*.nc")
# etc.
"""

nc_data_h0 = xr.open_mfdataset(h0_file_names,
                               combine='by_coords',
                               decode_times=True)
"""
nc_data_h1 = xr.open_mfdataset(h1_file_names,
                               combine='by_coords',
                               decode_times=True)
# etc.
"""

print(f"Time it took to read the data: {time.strftime('%Hh%Mm%Ss', time.gmtime(time.time()-start))}.")

---
First, investigate how time is stored in the resulting object.

In [ ]:
nc_data_h0["time"]

In [ ]:
# Cast time to datetimeindex for easier handling, e.g., when plotting. Throws a warning due to the special calendar format.
nc_data_h0["time"] = nc_data_h0.indexes["time"].to_datetimeindex()

In [ ]:
# Check how the updated format looks like
nc_data_h0["time"]

Note that the solution above will result in slight date mismatches between the model output and the plots. One of the reasons is that the GSWP3 climate input data uses a special calendar format, ['noleap'](http://cfconventions.org/cf-conventions/cf-conventions#calendar), where all years (including leap years) are represented with 365 days. To align the dates properly, we would need to resample the dates more carefully, but we will keep it simplified here.

In [ ]:
# Print full dataset information
nc_data_h0

If you want to learn more about the structure of NetCDF files, you may find [this introduction summary](https://adyork.github.io/python-oceanography-lesson/17-Intro-NetCDF/index.html) helpful.

---
# Create simple time series line plots
##### Plot variables with only time and lndgrid dimensions.
There are multiple ways to create output plots from xarray `DataArray` objects, i.e., for all the variables contained in the full xarray `DataSet` that we created when reading in the NetCDF files. For more details, check https://docs.xarray.dev/en/latest/user-guide/plotting.html.

Let's start by selecting one of the `Data variables` from the output above that only has `time` and `lndgrid` dimensions (listed in the parentheses in the second column when you display the data).

In [ ]:
var_name = "AGB" # "AREA_TREES", "NPP", ...

Next, explore your choice by assigning the object to a new Python variable and by printing its content.

In [ ]:
var = nc_data_h0[var_name]
var

---
The simplest way to create a plot is to use the `plot()` method that is already built into each `DataArray`. This approach is great for quick exploration, but it can arguably be a bit tricky to achieved desired customized the output.

In [ ]:
# Directly call the DataArray's plot method
var.plot()

As you see, this method automatically detects the internal structure of the data (2D-line plot) and directly assigns sensible values and labels to the axes (i.e., dates and variable long name including unit). It is also possible to provide additional arguments to the function call to adapt the plot, see [here](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.plot.html#xarray.DataArray.plot) for details.

However, it is also possible to create plots by accessing the object's values directly, e.g., with the popular `matplotlib` library. Note that the `DataArray.plot()` method also uses this library under the hood.

In [ ]:
# Create plot using matplotlib library directly
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(nc_data_h0['time'], var)
ax.set_title(f"Plotted variable: {var_name}")
ax.set_xlabel("Date")
ax.set_ylabel(f"{var.long_name} [{nc_data_h0[var_name].units}]")

🤔 **Question 1**:
Try to explain the plot for the variable that you picked.

---
# Plotting variables with more dimensions

In [ ]:
# Variable name for biomass per PFT in dataset
var_name = "PFTbiomass"

Start by again calling the internal `DataArray.plot()` method.

In [ ]:
pft_biomass = nc_data_h0[var_name]
pft_biomass.plot()

🤔 **Question 2**:
What is your impression about the output and how does it differ from the previous example? Do you think this is a good way to visualize the data?
---
The following cell defines some FATES PFT properties as Python `dict`(ionary) objects that we will use for plotting. Execute the code, but don't worry too much about the content.

In [ ]:
# Dictionary assigning long names and plot properties (NB: this is version and model setting dependent!)
# dict keys are the PFT indices used within the FATES output
default_pft_dict = {
    "1": {
        "long_name": "Broadleaf evergreen tropical tree",
        "group": "tree"
    },
    "2": {
        "long_name": "Needleleaf evergreen extratropical tree",
        "group": "tree"
    },
    "3": {
        "long_name": "Needleleaf cold-deciduous extratropical tree",
        "group": "tree"
    },
    "4": {
        "long_name": "Broadleaf evergreen extratropical tree",
        "group": "tree"
    },
    "5": {
        "long_name": "Broadleaf hydro-deciduous tropical tree",
        "group": "tree"
    },
    "6": {
        "long_name": "Broadleaf cold-deciduous extratropical tree",
        "group": "tree"
    },
    "7": {
        "long_name": "Broadleaf evergreen extratropical shrub",
        "group": "shrub"
    },
    "8": {
        "long_name": "Broadleaf hydro-deciduous extratropical shrub",
        "group": "shrub"
    },
    "9": {
        "long_name": "Broadleaf cold-deciduous extratropical shrub",
        "group": "shrub"
    },
    "10": {
        "long_name": "Arctic C3 grass",
        "group": "grass"
    },
    "11": {
        "long_name": "Cool C3 grass",
        "group": "grass"
    },
    "12": {
        "long_name": "C4 grass",
        "group": "grass"
    }
}

pft_group_plot_dict = {
    "tree": {
        "linestyle": "-"
    },
    "shrub": {
        "linestyle": "--"
    },
    "grass": {
        "linestyle": ":"
    }
}

<h3 style="color:red;">NB! Did you use non-default PFT indices?</h3>
If you changed the number of PFTs for your simulation, you need to adjust the following code. I.e., you need to enter the same list of PFT indices you used when you created the case in the GUI:

1. Open the GUI window/tab (localhost:8080) and click on the site name, either using the map or the overview at the top
2. Under `Cases`, search for the case ID you are analyzing here
3. Click on ___See Variables___
4. Copy the list of integers displayed for `included_pft_indices` into the `custom_pft_indices` variable in the code cell below
5. Set the value for `use_custom_pfts` to `True`

In [ ]:
# ADJUST HERE #
use_custom_pfts = True  # True or False
custom_pft_indices = [  # Add the comma-seperated list of PFT indices below. OBS! Do not remove the square brackets!
    7, 8, 9, 10, 11
]

# NO NEED TO TOUCH THE FOLLOWING LINES #
# Create new dict for included PFTs from default
custom_pft_dict = {str(idx+1): default_pft_dict[str(pft_idx)] for idx, pft_idx in enumerate(custom_pft_indices)}  
custom_pft_dict

In [ ]:
# Display variable details
pft_biomass

Let's now use Matplotlib again to create a multi-line plot, which hopefully depicts the PFT biomass dynamics in a more tangible way. Note that you can also achieve a similar result with the built-in `plot()` function (see documentation).

In [ ]:
# Instantiate plot
fig, ax = plt.subplots(figsize=(10,6))

# Create a multi-line plot by looping through PFT properties
plot_handle_list = []

# Determine whether default or customized PFTs were used based on boolean variable
pft_dict = custom_pft_dict if use_custom_pfts else default_pft_dict

for cur_pft_idx in pft_biomass.fates_levpft:
    
    cur_pft_dict = pft_dict[str(int(cur_pft_idx))]
    
    # Plot biomass for current PFT index
    cur_plot_handles, = ax.plot(nc_data_h0['time'],
                                pft_biomass.sel(fates_levpft=cur_pft_idx),
                                label=cur_pft_dict["long_name"],
                                linestyle=pft_group_plot_dict[cur_pft_dict["group"]]["linestyle"],
                                linewidth=1.5
                               )
    
    plot_handle_list.append(cur_plot_handles)

# Set overall plot layout
ax.set_title(f"{case_name}: {pft_biomass.long_name}")
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel(f"{var_name} [{pft_biomass.units}]", fontsize=14)

ax.legend(
    handles=plot_handle_list,
    bbox_to_anchor=(1.05, 1),  # Places the legend outside to the plotting area
    loc='upper left',
    fontsize=11
)

🤔 **Question 3**:
Discuss the plot. Does the amount of biomass and PFT distribution make sense to you?

---
# Run type: startup

After we have learned how to read the data and how to create output plots while taking the differences in variable dimensions into account, we now want to focus on explaining what we see. One notorious problem with earth system models is the computationally expensive **spin-up phase**, as you should remember from the LPJ-GUESS exercise. It can be maneageable for single-point simulations, but it gets more difficult the larger the simulated area is and the more model components you include. In CLM-FATES, The starting conditions are chiefly controlled by the `runtype` parameter, but depending on the setting, additional files and parameter changes can be required. Let's create plots to investigate some important state variables over our chosen simulation period, which in our protocol uses the `startup` setting.

_The following plots are inspired by the CTSM analysis tool mentioned [here](https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/users_guide/running-special-cases/Spinning-up-the-biogeochemistry-BGC-spinup.html)._

In [ ]:
# Use this cell to explore the variables defined below, if you want. E.g.:
# nc_data_h0["TWS"]

In [ ]:
import math

# Variables to plot from 'nc_data_h0', feel free to add more names to the list
var_names = ["GPP", "TWS", "TOTSOMC", "TLAI", "TOTVEGC"]

# Instantiate figure with subplot grid adapted to number of variables provided
fig, axes = plt.subplots(
    nrows=math.ceil(len(var_names)/2),
    ncols=2,
    figsize=(15, math.ceil(len(var_names)/2)*6)
)

# Delete empty axis object if number of variable names in list is odd
if len(axes.flatten()) != len(var_names):
    if len(axes.flatten()) == 2:
        fig.delaxes(axes[-1])
    else:
        fig.delaxes(axes[-1, -1])

# Plot all variables
for ax, var in zip(axes.flatten(), var_names):
    nc_data_h0[var].plot(ax=ax)

🤔 **Question 4**:
Is the ecosystem in your simulation in a steady state? Briefly discuss the individual plots.

🤔 **Question 5**:
Imagine you want to study many different model configurations at the same location while assuming the same environmental starting conditions. Do you have a suggestion how this could be achieved more efficiently? [Possible solution somewhere in here](https://usegalaxy.org/training-material/topics/climate/tutorials/fates/tutorial.html#setting-up-a-clm-fates-simulation).

---
# Carbon fluxes

There are many ways to analyze how carbon "moves" between the different components of the earth system model. Let's begin with the total, aggregated land surface fluxes in the single-point location over the simulated period.

In [ ]:
# Display data again to investigate variables related to C fluxes
nc_data_h0

In [ ]:
# Names of the variables that represent important C fluxes
# It is possible to add new variable names to the list (plot will be adjusted automatically)
c_flux_var_names = ["NEP", "GPP", "AR", "HR"]
c_flux_vars = [nc_data_h0[s] for s in c_flux_var_names]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

plot_handle_list = []

for c_var in c_flux_vars:
    cur_plot_handle, = ax.plot(c_var['time'], c_var, label=c_var.long_name.capitalize())
    plot_handle_list.append(cur_plot_handle)
    
# Add custom line as a test. Note that in python, array indexing starts at "0"! 
test_var = c_flux_vars[1] - c_flux_vars[2] - c_flux_vars[3]
test, = ax.plot(test_var["time"], test_var, 
                label=" - ".join(c_flux_var_names[1:4]),
                color="black",
                linewidth=3,
                linestyle=":",
                zorder=10  # Make sure line is on top of others
               )
plot_handle_list.append(test)

ax.set_title(f"{case_name}: aggregated C fluxes")
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel(f"Carbon flux [{c_flux_vars[0].units}]", fontsize=14)
# Use plain notation, i.e., avoid scientific "1e-3" notation
ax.ticklabel_format(style='plain', axis='y')

ax.legend(
    handles=plot_handle_list,
    bbox_to_anchor=(1.05, 1),
    loc='upper left',
    fontsize=14
)

🤔 **Question 6**:
The last line displayed in the legend was calculated as follows: `GPP - autotr. respiration - hetrotr. respiration`. Does it differ from the `NEP` values in your example? If it does, why could this be?

🤔 🐍 **Question 7**:
Add the output for the `NPP` history variable to the same plot. Also add a new line calculated as `GPP - autotr. respiration` (you can copy paste and adapt the previous example). What do you expect and what do you see?

---
# FATES patches
To better understand some important subgrid processes in the model, we will now analyze the output grouped by different properties of the modelled vegetation. First, we focus on the concepts of "age bins" and "patches". **NB**! The next code cells only work if you have successfully included the `NPATCH_BY_AGE` variable in the output for history tape 0 (`hist_fincl0` -> `NPATCH_BY_AGE`) as it is not included by default (see https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/master_list_fates.html).

🤔 **Question 8**:
Skim the following documentation: https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/DGVM/CLM50_Tech_Note_DGVM.html#the-representation-of-ecosystem-heterogeneity-in-fates. In this model, what is a patch? What is a cohort? What is a height class? How and why are patches and cohorts merged?

In [ ]:
npatch_by_age = nc_data_h0["NPATCH_BY_AGE"]
npatch_by_age

---
As usual, let's explore the standard plot first.

In [ ]:
npatch_by_age.plot()

In [ ]:
# Explore the 'fates_levage' variable
npatch_by_age.fates_levage

In [ ]:
# Make a list for labelling the legend in a more comprehensive way
vals = list(npatch_by_age.fates_levage.values)

age_labels = [str(int(vals[x]))+"-"+str(int(vals[x+1])) if x+1!=len(vals) else ">"+str(int(vals[x])) for x in range(len(vals))]
age_labels

🐍 _Python bonus question_ (optional): the second expression in the cell above is a so-called "list comprehension" and works for any number of "patch age levels", which is in fact an adjustable parameter in FATES. Rewrite the code using "classic" for-loop syntax, i.e.:
```
for x in y:
    # do something
```

#### Multi-line plot

In [ ]:
# Instantiate plot
fig, ax = plt.subplots(figsize=(10, 6))

# Create a multi-line plot by looping through PFT properties
plot_handle_list = []

for iter_idx, cur_age_idx in enumerate(npatch_by_age.fates_levage):
    
    #cur_pft_dict = pft_dict[str(int(cur_age_idx))]
    
    # Plot biomass for current PFT index
    cur_plot_handles, = ax.plot(nc_data_h0['time'],
                                npatch_by_age.sel(fates_levage=cur_age_idx),
                                label=age_labels[iter_idx],
                                #linestyle=pft_group_plot_dict[cur_pft_dict["group"]]["linestyle"],
                                linewidth=1.5
                               )
    
    plot_handle_list.append(cur_plot_handles)

# Set overall plot layout
ax.set_title(f"{case_name}: {npatch_by_age.long_name}")
ax.set_xlabel("Date", fontsize=14)
ax.set_ylabel(f"Number of patches", fontsize=14)

ax.legend(
    title="Age bin [years]",
    handles=plot_handle_list,
    bbox_to_anchor=(1.05, 1),  # Places the legend outside to the plotting area
    loc='upper left',
    fontsize=11
)

🤔 **Question 9**:
Explain the plot.

Also check out the plot below for an additional variable that represents how much area the different age classes account for in relation to the total area of patches.

In [ ]:
patch_area_by_age = nc_data_h0['PATCH_AREA_BY_AGE']

fig, ax = plt.subplots(figsize=(8, 8))

color_bar_plot = \
patch_area_by_age.T.plot(ax=ax,
                         cbar_kwargs={
                             'label': r'Patch Area by Age (m$^2$ patch m$^{-2}$ site)'
                             }
                        )


# Title and labels
ax.set_title(f"{case_name}: {patch_area_by_age.long_name}")
ax.set_ylabel("Age bin [years]", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.set_ylim(0, 20)  # We are only simulating 10 years and can ignore the higher age bins
ax.set_yticks([0, 5, 10, 15, 20])

---
# FATES PFT cohorts
🤔 **Question 9**: Create a suitable plot for the variable `M10_CACLS` - long_name : age senescence mortality by cohort age

---
# Integrating model input/output and observations

One notorious challenge with land surface models is to give them a stronger empirical foundation, i.e., to better link the model assumptions and output to available observational data. The list of difficulties and uncertainties regarding this endeavor is long and includes:

- Practical limitations for acquiring extensive datasets (cost, accessibility, cloud cover for remote sensing, ...)
- Harmonizing temporal and spatial scales (aggregated grid cell resolutions (km) vs. local observations (cm-m), ...)
- Different languages and foci of the scientific disciplines involved (biologists _vs._ geoscientists _vs._ computer scientists _vs._ ...)
- Data access and documentation
- etc.

Accordingly, a main goal of the NorESM modelling platform is to reduce the gap between ecological data collectors and the land surface modelling community. In addition to giving people easier access to running simulations, it aims at showcasing and facilitating how field observations can be used to both constrain the model input and validate its output. The `Vestland Climate Grid` sites you can choose within the GUI are associated with datasets from various ecological experiments at the respective locations. As a practical example, we will now extract some of their measurements for 'soil temperature' and see how it compares to the modelled counterpart in the CLM-FATES simulation. However, note that due to the time mismatch between the available observational data and the climate forcing data set we are providing in this version ([GSWP3](https://www.isimip.org/gettingstarted/details/4/)), we will not be able to compare the exact same points in time. 

## Example: soil temperature

In [ ]:
# Soil temperature history variable in CLM
var_name = "TSOI"

In [ ]:
t_soil = nc_data_h0[var_name]
t_soil

In [ ]:
# Explore the standard output plot to get a better feeling for the data
t_soil.plot()

In [ ]:
# Convert from Kelvin to Celsius
t_soil_celsius = t_soil - 273.15

In [ ]:
# Investigate the included ground levels. See:
# https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Ecosystem/CLM50_Tech_Note_Ecosystem.html#id17

t_soil_celsius.levgrnd

#### Exploring sources of uncertainty in the input data
By setting the `use_bedrock` flag to `True`, we are telling the model to use a variable soil depth rather than using a default value (e.g., remember how LPJ-GUESS Education uses a globally uniform 1.5 m soil column). See for example [this GH issue](https://github.com/NGEET/fates/pull/381) for additional information and associated discussions. The soil depth observations in the current version of the NorESM landsites platform input data were interpolated from a global soil map (see `input_visualization.ipynb` for details). Let's investigate if this data is appropriate for our study design.

In [ ]:
# Open the surface data file included in the standard input data
input_surfdata_dir = f"{case_name}_0.1.0_input"
surface_dset = xr.open_dataset(
    f'../../data/{case_id}/inputdata/lnd/clm2/surfdata_map/{case_name}/surfdata_{case_name}_simyr2000.nc'
)
# Print
surface_dset

Print information about the soil depth as contained in the default input data. More information [here](https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/tech_note/Ecosystem/CLM50_Tech_Note_Ecosystem.html#depth-to-bedrock).

In [ ]:
display(surface_dset['zbedrock'])
print("Default soil depth in surface data [m]: ", float(surface_dset['zbedrock']))

🤔 **Question 10**:
Do you think the soil depth value interpolated from a global map is realistic for your site and study setup?

This is an example how we can use _in situ_ observational data to constrain the model **input**: we could use soil depth measurements from the site to directly manipulate the input surface data NetCDF file.

---
In addition, even though our goal is to focus the "single-point" simulation to a somewhat homogenous, terrestrial, vegetated ecosystem, the default methods of creating the input data will also assign other land cover classes to the modelled locality. Explore the output below. Note that the values for percent PFT do not apply to our simulation as we used FATES. 

In [ ]:
for var_name in list(surface_dset.keys()):
    if var_name.startswith("PCT"):
        print(f"{var_name} - {surface_dset[var_name].long_name}:\n\n{surface_dset[var_name]}\n{'-'*70}\n")

Depending on the questions we seek to answer, we therefore also might need to further adjust the input data and model setup - or be at least aware of the limitations and uncertainties the forcing may introduce. Note that we plan to include optimized versions of the input data including different climate forcing sources in future releases of the platform. This is also a great example of how complicated the interdisciplinary communication between new users and model developers can be - there are many, many details that can impact your results!

---
Let's get back to the modelled soil temperature. Note how we here now combine the two previous ways of creating plots, thereby making use of `xarray`'s built-in convenience methods and direct manipulation of the `matplotlib` plot objects.

In [ ]:
# Define max depth to investigate top soil/ground layers
max_depth = 15  # [m]
bool_mask = t_soil_celsius.levgrnd < max_depth

# Infer integer index until which condition is met, warning: very unsafe approach that only works because data is ordered in a certain way
soil_idx = bool_mask.values.sum() - 1
# Subset data
t_soil_15m = t_soil_celsius.isel(levgrnd=slice(0, soil_idx))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

color_bar_plot = t_soil_15m.T.plot(ax=ax, 
                                   cbar_kwargs={
                                       'label': "Soil temperature [°C]"
                                   })

# Make plot more intuitive by setting depth=0 to the top
ax.invert_yaxis()

# Title and labels
ax.set_title(f"{case_name}: modelled soil and ground temperature")
ax.set_ylabel("Depth [m]", fontsize=14)
ax.set_xlabel("Date", fontsize=14)

🤔 **Question 11**: Describe and explain what you see. Do you think the results are realistic?

---

Next, we will load the observational data. Refer to https://osf.io/4c5v2/ for the full documentation and additional information. We can directly download the data from OSF into a DataFrame using the Python code below.

**Citation**: Vandvik, V., Telford, R. J., Halbritter, A. H., Jaroszynska, F., Lynn, J. S., Geange, S. R., … Rüthers, J. (2022, April 5). FunCaB - The role of functional group interactions in mediating climate change impacts on the Carbon dynamics and biodiversity of alpine ecosystems. Retrieved from osf.io/4c5v2

In [ ]:
# Read observational data as a pandas DataFrame. This step can take a while.
import pandas as pd

# Download data directly from OSF storage, make sure you are connected to the internet
obs_soil_t_df = pd.read_csv("https://osf.io/7tgxb/download")

In [ ]:
# Print unique site names in the data set
set(obs_soil_t_df["siteID"])

In [ ]:
# Map the name of the NorESM platform site codes to the corresponding name in the dataset
site_dict = {
    "ALP1": "Ulvehaugen",
    "ALP2": "Lavisdalen",
    "ALP3": "Gudmedalen",
    "ALP4": "Skjelingahaugen",
    "SUB1": "Alrust",
    "SUB2": "Hogsete",
    "SUB3": "Rambera",
    "SUB4": "Veskre",
    "BOR1": "Fauske",
    "BOR2": "Vikesland",
    "BOR3": "Arhelleren",
    "BOR4": "Ovstedalen"
}

In [ ]:
# Print some details about the data
print(obs_soil_t_df.shape)  # First number = number of rows (observations), second number = number of columns (variables)
obs_soil_t_df.head()

In [ ]:
# Subset soil temperatures for site
mysite_obs_soil_t_df = obs_soil_t_df[obs_soil_t_df["siteID"] == site_dict[case_name]]
mysite_obs_soil_t_df.head()

In [ ]:
# Print period when we have measurements available
print(min(mysite_obs_soil_t_df["date_time"]))
print(max(mysite_obs_soil_t_df["date_time"]))

In [ ]:
# Print inferred data types
mysite_obs_soil_t_df.dtypes

In [ ]:
# Next we need to calculate monthly means
df_monthly_mean = mysite_obs_soil_t_df.groupby(
    pd.PeriodIndex(mysite_obs_soil_t_df['date_time'], freq='M')
)['soiltemperature'].mean()

In [ ]:
# Plot monthly mean
pd.DataFrame(df_monthly_mean).plot()

## Modelled and observed soil T in one plot

In [ ]:
# Calculate mean for top 3 soil layers from model output to make it somewhat comparable
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

model_top_soil_t = t_soil_celsius.isel(levgrnd=slice(0, 3))
model_top_soil_t.plot(ax=ax1)
model_top_soil_t = model_top_soil_t.mean(dim="levgrnd")
model_top_soil_t.plot(ax=ax2)

ax1.set_title("Top three soil layers, soil T")
ax2.set_title("Mean top three soil layers, soil T")

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

# Plot modelled temperature at depth where measurements were taken, take average of top soil layers
ax.plot(model_top_soil_t['time'], model_top_soil_t, label="Modelled")
ax.plot(pd.DataFrame(df_monthly_mean), label="Observed")

ax.set_title(f"{case_name} - mean monthly soil temperatures")
ax.set_ylabel("Soil temperature [°C]", fontsize=14)
ax.set_xlabel("Date", fontsize=14)
ax.legend()
ax.grid()

🤔 **Question 12**: Discuss the results

---